In [1]:
#load jupyter extension that unables sql commands 
%load_ext sql

Establish connection with Database 

In [6]:
import csv, sqlite3

con = sqlite3.connect('my_data1.db')
cur = con.cursor()

In [7]:
#conntect to my_data1.db
%sql sqlite:///my_data1.db

In [8]:
import pandas as pd
df = pd.read_csv("Spacex.csv")
#push it into the database 
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [9]:
#remove blank rows from the database 
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

SQL queries 

In [31]:
#names of unique launch sites in the space mission
%sql select distinct Launch_Site from SPACEXTable

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [32]:
#Display 5 records where launch sites begin with the string 'CCA'
%sql select Launch_Site like 'CCA%' from SPACEXTable limit 5

 * sqlite:///my_data1.db
Done.


Launch_Site like 'CCA%'
1
1
1
1
1


In [33]:
#Display the total payload mass carried by boosters launched by NASA (CRS)
%sql select sum(PAYLOAD_MASS__KG_) AS 'Total_Payload' from Spacextable where customer LIKE 'NASA%'

 * sqlite:///my_data1.db
Done.


Total_Payload
99980


In [34]:
#Display average payload mass carried by booster version F9 v1.1
%sql select avg(PAYLOAD_MASS__KG_) as 'Average_Payload_version_F9_v1.1' from Spacextable where booster_version like 'F9 v1.1%'

 * sqlite:///my_data1.db
Done.


Average_Payload_version_F9_v1.1
2534.6666666666665


In [35]:
#List the date when the first succesful landing outcome in ground pad was acheived.
%sql select min(date) from Spacextable where Landing_Outcome like 'Success%'

 * sqlite:///my_data1.db
Done.


min(date)
2015-12-22


In [40]:
#List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
%sql select Booster_Version from spacextable where landing_outcome like 'Success%drone ship%' and PAYLOAD_MASS__KG_ > 4000 and PAYLOAD_MASS__KG_ < 6000

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [48]:
#List the total number of successful and failure mission outcomes
%sql select COUNT(CASE WHEN Mission_Outcome like 'Success%' THEN 1 END) AS Success_Count, COUNT(CASE WHEN Mission_Outcome like 'Failure%' THEN 1 END) AS Failure_Count from Spacextable

 * sqlite:///my_data1.db
Done.


Success_Count,Failure_Count
100,1


In [51]:
#List the names of the booster_versions which have carried the maximum payload mass. Use a subquery
%sql select booster_version, PAYLOAD_MASS__KG_ from Spacextable where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from Spacextable)

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


In [56]:
#List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015
#Note: SQLLite does not support monthnames. So you need to use substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.
%sql select substr(Date, 6,2), landing_outcome, booster_version, launch_site from spacextable where substr(Date,0,5)='2015' and landing_outcome like 'Failure%drone%'

 * sqlite:///my_data1.db
Done.


"substr(Date, 6,2)",Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [62]:
#Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.
%sql select landing_outcome, COUNT(*) AS landing_count from spacextable where date between '2010-06-04' and '2017-03-20' group by landing_outcome order by landing_count DESC

 * sqlite:///my_data1.db
Done.


Landing_Outcome,landing_count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
